In [ ]:
# stdlib
import os

# Testing works over 4 possibilities
# 1. (python/in-memory workers and using tox commands)
# 2. (python/in-memory workers and manually running notebooks)
# 3. (using k8s and using tox commands)
# 4. (using k8s and manually running notebooks)
# Uncomment the lines below if in the 4th possibility

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["CLUSTER_HTTP_PORT_HIGH"] = "9081"
# os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [ ]:
# stdlib
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
ROOT_EMAIL, ROOT_PASSWORD = "admin@bigquery.org", "bqpw"
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = os.environ.get("CLUSTER_HTTP_PORT_HIGH", "9081")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "9083")
num_jobs = int(os.environ.get("NUM_TEST_JOBS", 10))
print(environment, low_port)

In [ ]:
# stdlib

# third party
import pandas as pd

# syft absolute
import syft as sy
from syft.client.syncing import compare_clients
from syft.util.test_helpers.email_helpers import get_email_server
from syft.util.test_helpers.email_helpers import load_users
from syft.util.test_helpers.job_helpers import get_job_emails
from syft.util.test_helpers.job_helpers import get_request_for_job_info
from syft.util.test_helpers.job_helpers import load_jobs

# Launch server and login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

In [ ]:
email_server_low, smtp_server_low = get_email_server(server_side_type="low")
email_server_high, smtp_server_high = get_email_server(server_side_type="high")

In [ ]:
low_client = sy.login(
    url=f"http://localhost:{low_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)
high_client = sy.login(
    url=f"http://localhost:{high_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)

# Sync UserCode and Requests to High Side

In [ ]:
diffs = compare_clients(
    from_client=low_client, to_client=high_client, hide_usercode=False
)

In [ ]:
# check that only requests and usercode are in the diff
assert {diff.root_diff.obj_type.__qualname__ for diff in diffs.batches} == {
    "Request",
    "UserCode",
}

In [ ]:
widget = diffs.resolve()

widget._share_all()
widget._sync_all()

# Check that request synced over to high side

In [ ]:
len(high_client.code.get_all())

In [ ]:
assert len(high_client.code.get_all()) == num_jobs

In [ ]:
requests = high_client.requests.get_all_pending()
requests

In [ ]:
users = load_users(low_client)
jobs_data = load_jobs(users, low_client)
all_requests = high_client.requests
submitted_jobs_data = [job for job in jobs_data if job.is_submitted]
n_emails_per_job_user = {
    k: len(v)
    for k, v in get_job_emails(
        submitted_jobs_data, high_client, email_server_low
    ).items()
}

# Run or Deny

In [ ]:
submitted_jobs_data_should_succeed = [
    j for j in submitted_jobs_data if j.should_succeed
]
submitted_jobs_data_should_fail = [
    j for j in submitted_jobs_data if not j.should_succeed
]

In [ ]:
for job in submitted_jobs_data_should_succeed:
    request = get_request_for_job_info(all_requests, job)
    j = request.code(blocking=False)
    result = j.wait().get()
    assert isinstance(result, pd.DataFrame)
    job.admin_reviewed = True

In [ ]:
for job in submitted_jobs_data_should_fail:
    request = get_request_for_job_info(all_requests, job)
    response = request.deny(
        reason=f"Your request {job.func_name} looks wrong, try again."
    )
    assert isinstance(response, sy.SyftSuccess)
    assert not job.should_succeed
    job.admin_reviwed = True

# Sync job result to low side

In [ ]:
diffs = compare_clients(
    from_client=high_client, to_client=low_client, hide_usercode=False
)

In [ ]:
diffs.batches

In [ ]:
batch_root_strs = [x.root_diff.obj_type.__qualname__ for x in diffs.batches]
# for successful jobs, should be job, request, and usercode.
# For failed jobs, should be usercode, request only since job not run
expected_n_batches = 2 * len(submitted_jobs_data_should_fail) + 3 * len(
    submitted_jobs_data_should_succeed
)
assert len(diffs.batches) == expected_n_batches
assert "Job" in batch_root_strs

In [ ]:
widget = diffs.resolve()

widget._share_all()
widget._sync_all()

# Shutdown

In [ ]:
if environment != "remote":
    server_high.land()
    server_low.land()
smtp_server_low.stop()
smtp_server_high.stop()